<a href="https://colab.research.google.com/github/Vishal-113/RNN-and-NLP/blob/main/Implementing_an_RNN_for_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os

# 1. Load the dataset
path_to_file = tf.keras.utils.get_file("shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
text = open(path_to_file, 'rb').read().decode('utf-8')

# 2. Preprocessing
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Sequence preparation
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

dataset = sequences.map(split_input_target)

# 3. Create training batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# 4. Model building
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Input(batch_shape=(batch_size, None)),
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])


model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

# 5. Loss and compile
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# 6. Training
EPOCHS = 5
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5"),
    save_weights_only=True
)

model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 1375s 8s/step - loss: 2.8576
Epoch 2/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 1256s 7s/step - loss: 1.8525
Epoch 3/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 1297s 7s/step - loss: 1.5957
Epoch 4/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 1252s 7s/step - loss: 1.4699
Epoch 5/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 1291s 7s/step - loss: 1.3946
